In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [10]:
gd = [0.06, 0.223, 0.372, 0.196, -0.135, 0.426, 8.37]
gd_1 = [0.0508,1.1461,-0.3277,-0.2838,-0.4065,0.0431,8.48847328196] #X_1
gd_2 = [0.4539,-0.9662,-0.1322,-0.0,-0.7994,0.0441,8.12947125898] #X_2
gd_3 = [0.1704,-0.0947,-0.4962,-0.0,0.1037,0.0109,7.89770339493] #X_3
gd_4 = [0.8576,-0.8836,2.9865,0.0,-1.8508,-0.0143,8.33126119382] #X_4
gd_5 = [1.1957,-0.693,0.0005,-0.0,-0.6442,0.0293,8.21512214126] #X_5
ed = [-2.82, -1.104, 0.827, 0.96, 9.471]
ed_1 = [-2.147,-0.398,0.0,1.3791,9.6361887613] #X_1
ed_2 = [-0.4101,-0.9549,0.0,0.7409,8.82618121612] #X_2
ed_3 = [-1.7345,1.102,0.0,0.2559,7.82148744848] #X_3
ed_4 = [-3.9587,0.1358,0.8394,1.6839,9.56795015331] #X_4
ed_5 = [-1.8173,1.0672,0.0,-0.4283,9.10474754996] #X_5
gp = [-3.094, -7.545, 11.025, 9.011]
gp_1 = [-5.4733,-5.8343,10.7325,9.6022] #X_1
gp_2 = [-4.6902,-2.5154,16.9676,0.1287] #X_2
gp_3 = [-2.2335,-8.7922,6.2464,13.6984] #X_3
gp_4 = [-3.0828,-5.5514,10.5674,8.1592] #X_4
gp_5 = [-5.0835,-16.6975,7.5219,23.5545] #X_5

In [199]:
FEATURES = [
    "gd",
    "ed",
    "gp",
#    "isr",
#    "views", 
    "large", 
     "top", 
#    "speed_level",
#    "speed_level 1",
#    "speed_level 2",
#    "speed_level 3",
    "price"]

In [3]:
def get_speed_level(speed_level):
    if speed_level == 1:
        return  [1]
        #return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        return [0.4]
        #return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        return [0.1]
        #return [0,0,1]
        #print X[-1], speed_level
    else:
        return [0]
        #return [0,0,0]
        #print X[-1], speed_level

In [204]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        X+=[[]]
        #gd 0-7
        X[-1]+=[doc['degree'].count(u'Профессор')]
        X[-1]+=[int(doc['top'])]
        X[-1]+=[len(doc['illnesses'])]
        X[-1]+=[len(doc['languages'])]
        X[-1]+=[int(doc['h_index'])]
        X[-1]+=[int(doc['experience'])]
        X[-1]+=[1]
        #ed 7-12
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        X[-1]+=[len(list(set(ids_diad)))]
        X[-1]+=[len(doc['procedures']['diagnostics'])]
        X[-1]+=[len(list(set(ids_oper)))]
        X[-1]+=[len(doc['procedures']['operations'])]
        X[-1]+=[1]
        #gp 12-16
        X[-1]+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
        X[-1]+=[len(all_proc)] #gen_proc_cli
        X[-1]+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
        X[-1]+=[len(clinic['procedures']['operations'])] #gen_oper_cli
        #
        c_id = int(clinic['country_id'])
#        if c_id == 1:
#            X[-1]+=[1]
#        else:
#            X[-1]+=[0]
        #X[-1]+=[int(clinic['views'])]
        X[-1]+=[int(clinic['large'])]
        X[-1]+=[int(clinic['top'])]
        #X[-1]+=[get_speed_level(int(clinic['speed_level']))]
        #X[-1]+=[round(1-int(clinic_estim['price'])/10.0,4)]
        X[-1]+=[round(int(clinic['price_level'])/3.0,4)]
        
    y=[int(clinic_estim['rating'])] * len(all_doc)
    return [X, y]

In [5]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [6]:
def normale(mat):
    for i in range(0,len(mat[0])-1):
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [11]:
def pack(X, gd, ed, gp):
    new_X = []
    for x in X:
        new_X+= [sum([
            [round(sum(np.array(gd)*np.array(x[0:7])),5)],
            [round(sum(np.array(ed)*np.array(x[7:12])),5)],
            [round(sum(np.array(gp)*np.array(x[12:16])),5)],
            x[16:]
        ],[])]
    return new_X

In [16]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [ ]:
X, y = get_X_y_from(cancer_data)

In [205]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)
X_all=pack(X_all,gd, ed, gp)
normale(X_all)

normale(X_1)
X_1=pack(X_1,gd, ed, gp)
normale(X_1)

normale(X_2)
X_2=pack(X_2,gd, ed, gp)
normale(X_2)

normale(X_3)
X_3=pack(X_3,gd, ed, gp)
normale(X_3)

normale(X_4)
X_4=pack(X_4,gd, ed, gp)
normale(X_4)

normale(X_5)
X_5=pack(X_5,gd, ed, gp)
normale(X_5)

In [105]:
i=12
print cancer_data[i]['speed_level_name'], cancer_data[i]['speed_level']

>14 дней 3


In [ ]:
k=0
for x in X_all:
    if x[3]==x[4] and x[4] == 1:
        k+=1
        
print k, sum([i[4] for i in X_all])

In [207]:
for i in range(1,7):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

1
Total: 64, train: 57, test: 7
Residual sum of squares: 7.07
Train absolute: 2.13
Test absolute: 2.51
Absolute to mean: 39.10%
Train variance score: 0.34
Test variance score: -0.08

2
Total: 64, train: 51, test: 13
Residual sum of squares: 6.23
Train absolute: 2.15
Test absolute: 2.27
Absolute to mean: 37.90%
Train variance score: 0.30
Test variance score: 0.29

3
Total: 64, train: 44, test: 20
Residual sum of squares: 7.26
Train absolute: 2.13
Test absolute: 2.37
Absolute to mean: 38.15%
Train variance score: 0.35
Test variance score: 0.03

4
Total: 64, train: 38, test: 26
Residual sum of squares: 6.29
Train absolute: 2.21
Test absolute: 2.26
Absolute to mean: 35.69%
Train variance score: 0.36
Test variance score: 0.06

5
Total: 64, train: 32, test: 32
Residual sum of squares: 5.54
Train absolute: 2.32
Test absolute: 2.09
Absolute to mean: 34.06%
Train variance score: 0.34
Test variance score: 0.19

6
Total: 64, train: 25, test: 39
Residual sum of squares: 5.58
Train absolute: 2.35
T

In [208]:
regr = process_with(X_all, y_all, info=True, ts=0.2)

Total: 64, train: 51, test: 13
Residual sum of squares: 6.23
Train absolute: 2.15
Test absolute: 2.27
Absolute to mean: 37.90%
Train variance score: 0.30
Test variance score: 0.29


In [209]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0] #, i[1]
print regr.intercept_

Features sorted by their score:
6
19.138
10.4559
0.8319
1.3159
-0.1723
-4.0636
-19.8239322158


In [210]:
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

1
2
4
3
5
6


In [211]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_2)):    
    print "Real: %f \t Predicted: %f" %(y_2[i], regr.predict(X_2[i]))
    print X_2[i]
    print
    #print y_all[i]
    #print regr.predict(X_all[i])[0]

Real: 9.000000 	 Predicted: 7.968289
[0.9465, 1.0, 0.9469, 1.0, 1.0, 0.6667]

Real: 9.000000 	 Predicted: 7.251884
[0.9354, 0.9518, 0.9469, 1.0, 1.0, 0.6667]

Real: 9.000000 	 Predicted: 7.376999
[0.9244, 0.9839, 0.9469, 1.0, 1.0, 0.6667]

Real: 9.000000 	 Predicted: 6.901444
[0.9184, 0.9494, 0.9469, 1.0, 1.0, 0.6667]

Real: 7.000000 	 Predicted: 4.497540
[0.9261, 0.7522, 0.1516, 1.0, 0.0, 0.6667]

Real: 7.000000 	 Predicted: 4.655254
[0.8979, 0.8189, 0.1516, 1.0, 0.0, 0.6667]

Real: 7.000000 	 Predicted: 2.717968
[0.9031, 0.6241, 0.1516, 1.0, 0.0, 0.6667]

Real: 5.000000 	 Predicted: 8.040315
[0.9756, 0.9494, 1.0, 1.0, 1.0, 0.6667]

Real: 5.000000 	 Predicted: 7.300681
[0.9269, 0.9678, 1.0, 1.0, 1.0, 0.6667]

Real: 5.000000 	 Predicted: 6.755924
[1.0, 0.7819, 1.0, 1.0, 1.0, 0.6667]

Real: 5.000000 	 Predicted: 6.614747
[0.8998, 0.9518, 1.0, 1.0, 1.0, 0.6667]

Real: 5.000000 	 Predicted: 7.512320
[0.9467, 0.9518, 1.0, 1.0, 1.0, 0.6667]

Real: 5.000000 	 Predicted: 5.802840
[0.9225, 0.8

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    k=0
    for X in [X_1, X_2, X_3, X_4, X_5]:
        k += 1
        plt.plot(sorted([i[f] for i in X]))
        #plt.show()
        plt.savefig('%s - X%d.png' % (feature, k))
        plt.clf()
    f+=1

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    mass = [i[f] for i in X_all]
    k=0
    r=0
    l=0
    for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
        l=r
        r+=length
        k+=1
        plt.plot(sorted(mass[l:r]), label="X%d"%k)
        plt.ylim([min(mass),1])
        legend = plt.legend(loc=4, shadow=True)
        #plt.show()
    plt.savefig('%s.png' % (feature))
    plt.clf()
    f+=1

In [ ]:
plt.plot([np.linalg.norm(i) for i in X_all], color='black')
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    mass = [i[f] for i in X_all]
    k=0
    r=0
    l=0
    for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
        l=r
        r+=length
        k+=1
        plt.plot(y_all[l:r], [i[f] for i in X_all[l:r]], label="X%d"%k)
        plt.ylim([min(mass),1])
        plt.xlim([0,11])
        legend = plt.legend(loc=4, shadow=True)
        plt.savefig('%s - X%d.png' % (feature, k))
        plt.show()
        plt.clf()
    f+=1

In [ ]:
k=0
r=0
l=0
for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
    l=r
    r+=length
    k+=1
    plt.plot(y_all[l:r], [np.linalg.norm(i) for i in X_all[l:r]], label="X%d"%k)
    legend = plt.legend(loc=4, shadow=True)
    plt.xlim([0,11])
    plt.savefig('norms - X%d.png'%k)
    plt.clf()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)

In [ ]:
x = [i[-2] for i in X_all]
y_top = []
y_n_top = []
for l in range(0, len(x)):
    if x:
        y_top += [y[l]]
    else:
        y_n_top += [y[l]]
#for l in range(0, len(x)):
    #print x[l], y[l]

In [ ]:
for i in range(1,11):
    print i, y_top.count(i)

In [ ]:
FEATURES[-3]

In [ ]:
plt.plot( x, y_all, 'ro')
plt.show()

In [189]:
X_all[20]

[0.9756, 0.9695, 1.0, 1.0, 1.0, 0.0]

In [214]:
res_X = []
res_y = []
for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            print res_X[-1], res_y[-1]
        except Exception, err:
            print err, i, indices
            pass

8.00545551804 10
5.03071707742 9
4.53536628722 8
min() arg is an empty sequence 7 []
7.97594430692 6
min() arg is an empty sequence 5 []
0.280267397741 4
1.3423883102 3
min() arg is an empty sequence 2 []
3.66191102846 1
min() arg is an empty sequence 10 []
7.96828910082 9
min() arg is an empty sequence 8 []
4.65525404656 7
min() arg is an empty sequence 6 []
8.04031463375 5
4.10998814972 4
min() arg is an empty sequence 3 []
2.9752496328 2
min() arg is an empty sequence 1 []
min() arg is an empty sequence 10 []
5.12022239232 9
6.93378906135 8
min() arg is an empty sequence 7 []
7.94371081632 6
8.06206537555 5
3.41787752356 4
min() arg is an empty sequence 3 []
3.60537019264 2
3.53307174561 1
7.6373538272 10
5.63208360263 9
5.30055036299 8
5.48315875542 7
3.64667066033 6
5.29021388148 5
3.86815404647 4
5.55218748853 3
7.96828910082 2
3.78752321488 1
8.2631734085 10
6.68777775339 9
2.65588658924 8
min() arg is an empty sequence 7 []
6.78173215527 6
min() arg is an empty sequence 5 []
mi

In [216]:
for x in res_X:
    print x

8.00545551804
5.03071707742
4.53536628722
7.97594430692
0.280267397741
1.3423883102
3.66191102846
7.96828910082
4.65525404656
8.04031463375
4.10998814972
2.9752496328
5.12022239232
6.93378906135
7.94371081632
8.06206537555
3.41787752356
3.60537019264
3.53307174561
7.6373538272
5.63208360263
5.30055036299
5.48315875542
3.64667066033
5.29021388148
3.86815404647
5.55218748853
7.96828910082
3.78752321488
8.2631734085
6.68777775339
2.65588658924
6.78173215527
3.58563904306
2.91474566408


In [206]:
set([i[-1] for i in X_all])

{0.0, 0.3333, 0.6667, 1.0}

In [ ]:
y_all[13+17:13+17+12]

In [149]:
indices = [i for i, x in enumerate(y_1) if x == 2]
print indices
for ind in indices:
    print regr.predict(X_1[ind])

print max(regr.predict(X_1[min(indices):max(indices)+1]))

[]


ValueError: min() arg is an empty sequence